# Importando bibliotecas e dataset 

In [6]:
!python -m spacy download pt_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 602.7 kB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [1]:
import re
import string
import pandas as pd
import numpy as np
import spacy
import nltk
import pt_core_news_lg
from textblob import TextBlob
from spellchecker import SpellChecker
from spacy.lang.pt.stop_words import STOP_WORDS
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
# from nltk.sentiment import SentimentIntensityAnalyzer
from LeIA import SentimentIntensityAnalyzer
from torch.utils.data import DataLoader, SequentialSampler
import torch
from tqdm import tqdm

In [3]:
file_path = '../data/raw/ErikakHilton-tweets.csv'
df = pd.read_csv(file_path, on_bad_lines='skip', sep=';', encoding='utf-8')

/tmp/ipykernel_5955/4045662701.py:2: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, on_bad_lines='skip', sep=';', encoding='utf-8')


In [4]:
df.shape

(22505, 49)

# Filtrar tweets?
Pensei em filtrar os tweets, deixando somente os tweets direcionados a Erika, e para remover duplicatas.
Nessa fase não apliquei somente pra termos mais tweets para análise

In [10]:
# df[df['user'] != "https://twitter.com/ErikakHilton"].info()

# 1.Limpeza

In [11]:
# Removendo "não" das stopwords
STOP_WORDS.remove('não')

# Instanciando spacy/pt_core_news_lg - corpus completo
nlp = spacy.load('pt_core_news_lg')

In [12]:
tweets = df['rawContent']

In [13]:
tweets.sample(10)

9900     @ErikakHilton Bolsonaro não é honesto coisa ne...
3620     @ErikakHilton Faz isso não, na época que ele f...
18066    @ErikakHilton @LulaOficial @jairbolsonaro O FA...
19879    @ErikakHilton vamos ter representatividade no ...
7694                                    @ErikakHilton Fake
2672                         @ErikakHilton @BaixaEssaPorra
13467                        @ErikakHilton PINTOU UM CLIMA
21901    @ErikakHilton @visoesoniricas eu n aguento td ...
18227             @ErikakHilton @camaradeputados Que ótimo
14976                               @ErikakHilton @S1n1nh4
Name: rawContent, dtype: object

In [14]:
# Função para remover URL
def remover_url(texto):
    texto = re.sub(r'http\S+', '', texto)
    return texto

In [15]:
# Função para remover contas
def remover_twiters(texto):
    texto = re.sub(r'@\w+', '', texto)
    return texto

In [16]:
# Função para remover pontuações
def remover_pontuacao(texto):
    texto = re.sub(f"[{re.escape(string.punctuation)}]", "", texto)
    return texto

In [17]:
# Aplicando limpezas
tweets = tweets.str.lower()
tweets = tweets.apply(remover_url)
tweets = tweets.apply(remover_twiters)
tweets = tweets.apply(remover_pontuacao)

In [18]:
tweets.sample(10)

11821                                                     
13847     o que sabemos  e sempre soubemos  você era um...
7754                                          cuteeeeeeeee
12198    levantamento divulgado hoje 1910 ouviu 2912 pe...
21989                               lula presidente amanhã
17203                                              nunca\n
2649                                               1️⃣3️⃣ 
6790                                                      
8916      eles estão desesperados\neles estão desespera...
13555                                   quem quer dinheiro
Name: rawContent, dtype: object

In [19]:
# Função para remoção de stopwords e digitos
def limpar_texto(texto):
    doc = nlp(texto)
    tokens = [token.text for token in doc if not token.is_stop and not token.is_digit]
    texto_limpo = " ".join(tokens)
    return texto_limpo

In [20]:
tweets = tweets.apply(limpar_texto)

# Tentativa de corrigir typos

Com essa biblioteca seria possivel corrigir erros de digitação.<br>
Em alguns tweets percebi que tinham palavras com letras repetidas ou contrações e isso poderia dificutar a nossa análise.<br>
Mas não consegui aplicar essa etapa da limpeza. Vou deixar marcado caso alguem queira tentar aplicar.

In [21]:
# Instanciando spellchecker
# spell = SpellChecker(language='pt')

In [22]:
# Função para correção de grafia
# def corretor_grafia(texto):
#     tb = TextBlob(texto)
#     corrected_words = []
#     for word in tb.words:
#         corrected_word = word.correct()
#         corrected_words.append(corrected_word)
#     return ' '.join(corrected_words)

In [23]:
# df_tweets = df_tweets.apply(corretor_grafia)

In [24]:
# Transformando em dataframe
df_tweets = tweets.to_frame()

In [25]:
# Identificando e excluindo linhas vazias. 
# Aqui não foi possível excluir nem identificar com funções NA, pq a linhas estava vazia.
df_tweets = df_tweets[df_tweets['rawContent'] != " "]

In [26]:
df_tweets

,rawContent
0,longo dia recebendo denúncias pessoas presas h...
1,simplesmente ❤ ️ ❤ ️ ❤ ️
2,hahahhaha 💜 💜 💜
3,longo dia recebendo denúncias pessoas presas h...
5,🫶🏾🫶🏾
...,...
22500,casa vc opções mãe tias candidatas saí...
22501,sairão ♡
22502,candidatos pra estadual federal respectivamente
22503,força fabio 💜


In [27]:
# Função para lemmatização
def lemmatiza(texto):
    doc = nlp(texto)
    lemmatized_tokens = []
    for token in doc:
        lemmatized_tokens.append(token.lemma_)
    return " ".join(lemmatized_tokens)

In [28]:
df_tweets['tweets_lemma'] = df_tweets['rawContent'].apply(lemmatiza)

In [29]:
df_tweets

,rawContent,tweets_lemma
0,longo dia recebendo denúncias pessoas presas h...,longo dia receber denúncia pessoa presas haver...
1,simplesmente ❤ ️ ❤ ️ ❤ ️,simplesmente ❤ ️ ❤ ️ ❤ ️
2,hahahhaha 💜 💜 💜,hahahhaha 💜 💜 💜
3,longo dia recebendo denúncias pessoas presas h...,longo dia receber denúncia pessoa presas haver...
5,🫶🏾🫶🏾,🫶🏾🫶🏾
...,...,...
22500,casa vc opções mãe tias candidatas saí...,casa vc opção mãe tia candidato sair...
22501,sairão ♡,sair ♡
22502,candidatos pra estadual federal respectivamente,candidato pra estadual federal respectivamente
22503,força fabio 💜,força Fabio 💜


In [30]:
# Função para classificação de palavras
def pos_tag(texto):
    doc = nlp(texto)
    pos_tags = []
    for token in doc:
        pos_tags.append((token.text, token.pos_))
    return pos_tags

In [31]:
df_tweets['tweets_pos'] = df_tweets['rawContent'].apply(pos_tag)

In [32]:
df_tweets['tweets_pos']

0        [(longo, ADJ), (dia, NOUN), (recebendo, VERB),...
1        [(  , SPACE), (simplesmente, ADV), (❤, NOUN), ...
2        [(  , SPACE), (hahahhaha, NOUN), (💜, PUNCT), (...
3        [(longo, ADJ), (dia, NOUN), (recebendo, VERB),...
5                              [(  , SPACE), (🫶🏾🫶🏾, NOUN)]
                               ...                        
22500    [(      , SPACE), (casa, NOUN), (vc, PROPN), (...
22501         [(      , SPACE), (sairão, VERB), (♡, NOUN)]
22502    [(candidatos, NOUN), (pra, ADP), (estadual, NO...
22503    [(   , SPACE), (força, NOUN), (fabio, PROPN), ...
22504    [(   , SPACE), (ownnn, NOUN), (conte, VERB), (...
Name: tweets_pos, Length: 21357, dtype: object

# Pipeline Completo
Para facilitar e aplicar todas as transformações e limpezas foi proposto pipeline, porém ele ainda não funciona.
Caso queiram aplicar mandem bala. Em uma das conversas com ChatGPT uma hipotese levantada foi de que o pipeline não suporta emojis e caracteres especial. Então precisariamos de uma limpeza maior. Ai entra outro ponto, se tirarmos todos esses caracteres podemos processar com mais eficacia, porém perderiamos informações no meio.

In [33]:
def pos_tagging(text):
    doc = nlp(text)
    pos_tags = []
    for token in doc:
        pos_tags.append(token.pos_)
    return pos_tags

def lemmatize(text):
    doc = nlp(text)
    lemmatized_tokens = []
    for token in doc:
        lemmatized_tokens.append(token.lemma_)
    return " ".join(lemmatized_tokens)

def lemmatize_texts(texts):
    return texts.apply(lemmatize)

def create_tfidf_matrix(texts):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(texts)
    return tfidf_matrix

In [34]:
pipeline = Pipeline([
    # ('pos_tagging', FunctionTransformer(pos_tagging)),
    ('lemmatization', FunctionTransformer(lemmatize_texts)),
    ('tfidf', FunctionTransformer(create_tfidf_matrix)),
])

In [35]:
# preprocessed_data = pipeline.fit_transform(df_tweets['rawContent'])

In [36]:
# preprocessed_data

In [37]:
df_tweets['pos_tagged'] = df_tweets['rawContent'].apply(pos_tagging)
df_tweets['lemma'] = df_tweets['pos_tagged'].apply(lemmatize_text)
df_tweets['tfidf_matrix'] = df_tweets['lemma'].apply(create_tfidf_matrix)

NameError: name 'lemmatize_text' is not defined

# Modelos não supervisionados

## Clustering-based Approach (CBA)

In [43]:
# Tokenizar e preprocessar
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df_tweets['tweets_lemma'])

# Treinar cluster para agrupar tweets similares
k = 2  # numero de clusters
kmeans = KMeans(n_clusters=k)
labels = kmeans.fit_predict(X)

# Rotular (não entendi exatamente porque e como estipularam 0 para positivo)
df_tweets['classif_cba'] = ['positive' if label == 0 else 'negative' for label in labels]

/home/paim/projects/projeto-tera/venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [44]:
df_tweets['classif_cba'].value_counts()

positive    20379
negative      978
Name: classif_cba, dtype: int64

## Topic Modeling-based Approach (TMBA)

In [46]:
# Tokenizar e preprocessar
cv = CountVectorizer()
X = cv.fit_transform(df_tweets['tweets_lemma'])

# Treinar LDA para identificar topicos/temas
n_topics = 2  # numero de topicos
lda = LatentDirichletAllocation(n_components=n_topics)
lda.fit(X)

# Manualmente atribuir rotulos para cada topico
topic_sentiments = ['positive', 'negative']
topic_labels = [topic_sentiments[i] for i in lda.transform(X).argmax(axis=1)]

# Rotular tweets baseado em topicos
df_tweets['classif_tmba'] = topic_labels

In [47]:
df_tweets['classif_tmba'].value_counts()

positive    12969
negative     8388
Name: classif_tmba, dtype: int64

## Sentiment Intensity Analysis (SIA)

In [ ]:
nltk.download('vader_lexicon') # download do sentiment lexicon

In [48]:
from nltk.sentiment import SentimentIntensityAnalyzer
# Não é adequado para o portugues, mas existem adaptações como LeIA 
# https://github.com/rafjaa/LeIA

sia = SentimentIntensityAnalyzer() # criar sentiment analyzer

In [49]:
# Loop pelos tweets e retornar score de sentimento
sentiments = []

for tweet in df_tweets['tweets_lemma']:
    sentiment = sia.polarity_scores(tweet)
    sentiments.append(sentiment)

In [50]:
df_tweets['sentiment_sia'] = sentiments

In [62]:
df_tweets['sentiment_sia'].value_counts()

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}            18897
{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}             1447
{'neg': 0.608, 'neu': 0.392, 'pos': 0.0, 'compound': -0.4767}       33
{'neg': 0.0, 'neu': 0.333, 'pos': 0.667, 'compound': 0.6124}        27
{'neg': 0.241, 'neu': 0.759, 'pos': 0.0, 'compound': -0.7906}       25
                                                                 ...  
{'neg': 0.0, 'neu': 0.545, 'pos': 0.455, 'compound': 0.3612}         1
{'neg': 0.0, 'neu': 0.517, 'pos': 0.483, 'compound': 0.4215}         1
{'neg': 0.0, 'neu': 0.233, 'pos': 0.767, 'compound': 0.5106}         1
{'neg': 0.0, 'neu': 0.819, 'pos': 0.181, 'compound': 0.4939}         1
{'neg': 0.0, 'neu': 0.68, 'pos': 0.32, 'compound': 0.5106}           1
Name: sentiment_sia, Length: 418, dtype: int64

## LeIA (Léxico para Inferência Adaptada)

In [52]:
from LeIA import SentimentIntensityAnalyzer as LeIASentimentIntensityAnalyzer

sia = LeIASentimentIntensityAnalyzer() # criar sentiment analyzer

In [53]:
# Loop pelos tweets e retornar score de sentimento
sentiments_leia = []

for tweet in df_tweets['tweets_lemma']:
    sentiment = sia.polarity_scores(tweet)
    sentiments_leia.append(sentiment)

In [54]:
df_tweets['sentiment_sia_leia'] = sentiments_leia

In [55]:
df_tweets['sentiment_sia_leia'].value_counts()

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}              11014
{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}               1540
{'neg': 0.423, 'neu': 0.577, 'pos': 0.0, 'compound': -0.296}         227
{'neg': 0.0, 'neu': 0.345, 'pos': 0.655, 'compound': 0.5859}         196
{'neg': 0.524, 'neu': 0.476, 'pos': 0.0, 'compound': -0.296}         155
                                                                   ...  
{'neg': 0.384, 'neu': 0.434, 'pos': 0.182, 'compound': -0.7906}        1
{'neg': 0.25, 'neu': 0.53, 'pos': 0.22, 'compound': -0.1027}           1
{'neg': 0.191, 'neu': 0.601, 'pos': 0.208, 'compound': 0.0772}         1
{'neg': 0.231, 'neu': 0.769, 'pos': 0.0, 'compound': -0.765}           1
{'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound': -0.0516}              1
Name: sentiment_sia_leia, Length: 3364, dtype: int64

In [67]:
pd.set_option('display.max_colwidth', 100)
df_tweets.sample(10)[['tweets_lemma', 'sentiment_sia_leia']]

,tweets_lemma,sentiment_sia_leia
3572,bolsonaro mentir pra agradecer alguém vacinar sentido agradecer luta João doria \n\n ser bolson...,"{'neg': 0.082, 'neu': 0.783, 'pos': 0.135, 'compound': 0.3612}"
15349,tarcísio freitas gostar vender técnico carregar mau fanatismo bolsonarista importar Paulo pensam...,"{'neg': 0.154, 'neu': 0.752, 'pos': 0.094, 'compound': -0.1531}"
11498,hoje sair notícia posto ipiranga pensar mudar aumento salário mínimo passar inflação prever a...,"{'neg': 0.124, 'neu': 0.876, 'pos': 0.0, 'compound': -0.34}"
8912,passe livre alguém pagar conta adivinho 🤡,"{'neg': 0.0, 'neu': 0.571, 'pos': 0.429, 'compound': 0.25}"
17338,kkkk,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
17923,Lula vencer turno milhão voto estar forte apoio parte país assista programa turno ir frente bras...,"{'neg': 0.079, 'neu': 0.789, 'pos': 0.132, 'compound': 0.25}"
16338,falar voo não pq tarcísi0 capaz cair ponte aéreo \n haddadprontoprasp haddadgovernadorsp1️⃣3️⃣,"{'neg': 0.159, 'neu': 0.652, 'pos': 0.188, 'compound': 0.1027}"
8256,propor região 📍 ribeirão \n vote 1️⃣3️⃣ ✅,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
19141,Gataaaa,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
834,ir ganhar erika 🙏 🏼 🙏 🏼 🙏 🏼 querer prejudicar nordeste hoje gente resistir,"{'neg': 0.0, 'neu': 0.678, 'pos': 0.322, 'compound': 0.5859}"


In [68]:
df_tweets.shape

(21357, 6)

# Modelos supervisionados

## Sentiment140

In [56]:
# Função para classificação usando sentiment140
def classif_sent140(tweet):
    # Não é preparado para portugues
    analise = TextBlob(tweet)
    if analise.sentiment.polarity > 0:
        return 'positivo'
    elif analise.sentiment.polarity == 0:
        return 'neutro'
    else:
        return 'negativo'

In [57]:
df_tweets['classif_sent140'] = df_tweets['tweets_lemma'].apply(classif_sent140)

In [58]:
df_tweets['classif_sent140'].value_counts()

neutro      20191
positivo      927
negativo      239
Name: classif_sent140, dtype: int64

## BERTimbau

In [61]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Carregar tokenizer
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

# Carregar modelo pré treinado BERTimbau
model = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=2)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

In [66]:
#atribuir análise ao CPU
device = torch.device('cpu')
# device = torch.device('cuda')

# Função para analise sentimento
def predict_sentiment(tweet):
    # Encode do tweet com tokenizer do BERT
    inputs = tokenizer.encode_plus(tweet, add_special_tokens=True, return_tensors="pt")
    # Input IDs and attention mask (não entendi exatamente essa parte)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    # Fazer previsão com BERTimbau
    outputs = model(input_ids, attention_mask=attention_mask)
    # Rotulos de classificação (positivo ou negativo)
    predicted_label = torch.argmax(outputs[0]).item()
    # Returnar rotulos previstos
    return "positivo" if predicted_label == 1 else "negativo"


In [67]:
# Lista vazia para armazenar classificação de cada tweet
predicted_sentiments = []

# Loop por cada tweet e previsão
for tweet in df_tweets['tweets_lemma']:
    predicted_sentiment = predict_sentiment(tweet)
    predicted_sentiments.append(predicted_sentiment)

In [68]:
df_tweets["classif_bertimbau"] = predicted_sentiments

In [69]:
df_tweets["classif_bertimbau"].value_counts()

positivo    21169
negativo      188
Name: classif_bertimbau, dtype: int64

In [70]:
df_tweets.rename(columns={'predicted_sentiment':'classif_bertimbau'}, inplace=True)

Nessa linha abaixo eu quis criar uma coluna com o compound da classificação SIA, pois o resultado do modelo é um dicionário com os valores que o modelo calculou para cada sentimento(neutro, positivo e negativo, sendo o compound uma metrica relativa aos 3 sentimentos presentes no texto)

In [98]:
list = []

for result in df_tweets['sentiment_sia']:
    compound = result['compound']
    list.append(compound)

SyntaxError: invalid syntax (4004766246.py, line 1)

In [82]:
df_tweets['compound_sia'] = list

In [91]:
df_tweets.rename(columns={'sentiment_sia':'classif_sia'}, inplace=True)

In [93]:
# Slavando resultado
df_tweets.to_csv('data/df_tweets_classif.csv', index_label=False)

In [94]:
# Carregando dataframe
df_novo = pd.read_csv('data/df_tweets_classif.csv')

In [95]:
df_novo

,rawContent,tweets_lemma,tweets_pos,classif_cba,classif_tmba,classif_sia,classif_sent140,classif_bertimbau,compound_sia
0,longo dia recebendo denúncias pessoas presas h...,longo dia receber denúncia pessoa presas haver...,"[('longo', 'ADJ'), ('dia', 'NOUN'), ('recebend...",positive,negative,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutro,negative,0.0000
1,simplesmente ❤ ️ ❤ ️ ❤ ️,simplesmente ❤ ️ ❤ ️ ❤ ️,"[(' ', 'SPACE'), ('simplesmente', 'ADV'), ('❤...",positive,positive,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutro,negative,0.0000
2,hahahhaha 💜 💜 💜,hahahhaha 💜 💜 💜,"[(' ', 'SPACE'), ('hahahhaha', 'NOUN'), ('💜',...",positive,positive,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutro,negative,0.0000
3,longo dia recebendo denúncias pessoas presas h...,longo dia receber denúncia pessoa presas haver...,"[('longo', 'ADJ'), ('dia', 'NOUN'), ('recebend...",positive,negative,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutro,negative,0.0000
5,🫶🏾🫶🏾,🫶🏾🫶🏾,"[(' ', 'SPACE'), ('\U0001faf6🏾\U0001faf6🏾', '...",positive,positive,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutro,negative,0.0000
...,...,...,...,...,...,...,...,...,...
22500,casa vc opções mãe tias candidatas saí...,casa vc opção mãe tia candidato sair...,"[(' ', 'SPACE'), ('casa', 'NOUN'), ('vc',...",positive,positive,"{'neg': 0.0, 'neu': 0.68, 'pos': 0.32, 'compou...",neutro,negative,0.5106
22501,sairão ♡,sair ♡,"[(' ', 'SPACE'), ('sairão', 'VERB'), ('♡'...",positive,positive,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutro,negative,0.0000
22502,candidatos pra estadual federal respectivamente,candidato pra estadual federal respectivamente,"[('candidatos', 'NOUN'), ('pra', 'ADP'), ('est...",positive,positive,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutro,negative,0.0000
22503,força fabio 💜,força Fabio 💜,"[(' ', 'SPACE'), ('força', 'NOUN'), ('fabio'...",positive,negative,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutro,negative,0.0000
